In [2]:
import bs4
import pandas as pd

In [ ]:
from urllib.request import urlopen as uReq

In [ ]:
from bs4 import BeautifulSoup as soup

In [ ]:
topic_list = list()
author_list = list()
date_list = list()
replies_list = list()
stats_list = list()
forum_page_list = list()
page_topic_count_list = list()
topic_url_list = list()
firstposts_list = list()
initial_post_username_list = list()
initial_post_date_list = list()
from datetime import datetime
topic_by_author = pd.DataFrame()

In [ ]:
my_url = f'https://www.celiac.com/forums/forum/13-celiac-disease-pre-diagnosis-testing-symptoms/page/3/?sortby=start_date&sortdirection=desc'
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html, "html.parser")
final_page = int(page_soup.find("li",{"class":"ipsPagination_last"}).a.get("data-page"))

In [ ]:
start_page = 53
final_page = 103
end_page = final_page+1
total_pages_evaluated = 0
for forum_page in range(start_page,end_page):
    my_url = f'https://www.celiac.com/forums/forum/13-celiac-disease-pre-diagnosis-testing-symptoms/page/{forum_page}/?sortby=start_date&sortdirection=desc'
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    
    topic_containers =  page_soup.findAll("span",{"class":"ipsType_break ipsContained"})
    author_containers = page_soup.findAll("div",{"class":"ipsDataItem_meta ipsType_reset ipsType_light ipsType_blendLinks"})
    datetime_containers = page_soup.findAll("a",{"class":"ipsType_blendLinks"})
    replies_containers = page_soup.findAll("li",{"data-stattype":"forums_comments"}) #renamed to comments since this includes original post as well as replies.
    span_containers = page_soup.findAll("span",{"class":"ipsType_break ipsContained"})
    num_views_containers = page_soup.findAll("li",{"data-stattype":"num_views"})
    
    
    page_topic_count = 0
    for topic_container, topic_url_container, author_container, replies_container, num_views_container, datetime_container in zip(topic_containers, topic_containers, author_containers, replies_containers, num_views_containers, datetime_containers):
       
        topic = topic_container.text.strip()
        author = author_container.a.text.strip()
        datestr = datetime_container.time.text.strip()
        
        replies_str = replies_container.span.text.strip()
        replies_k = replies_str.__contains__('k')
        replies = int(float(replies_str.replace('k',''))*1000) if replies_k else int(float(replies_str))
        
        num_views_str = num_views_container.span.text.strip()
        num_views_k = num_views_str.__contains__('k')
        num_views = int(float(num_views_str.replace('k',''))*1000) if num_views_k else int(float(num_views_str))
        
        topic_url = topic_url_container.find("a").get("href")
        
        #below sourced from chatGPT
        try:
            # Try parsing with year (%Y) format
            date_obj = datetime.strptime(datestr, '%B %d, %Y')
        except ValueError:
            # If year is not present, add a default year
            date_obj = datetime.strptime(datestr + ', 2023', '%B %d, %Y')
        #above sourced from chatGPT
    

        uClient2 = uReq(topic_url)
        page_html2 = uClient2.read()
        uClient2.close()
        page_soup2 = soup(page_html2, "html.parser")
        posts = page_soup2.findAll("div",{"class":"cPost_contentWrap"})
        initial_post_p = posts[0].find_all('p')
        initial_post = ' '.join([text.get_text(strip=True) for text in initial_post_p])
        initial_post_username = page_soup2.find("p",{"class":"ipsType_reset ipsType_blendLinks"}).a.text
        initial_post_dt = page_soup2.find("p",{"class":"ipsType_reset ipsType_blendLinks"}).time.get("datetime")[0:10]
        initial_post_dt_format = datetime.strptime(initial_post_dt, '%Y-%m-%d')
        initial_post_date = initial_post_dt_format.date()

        
        last_updated = date_obj.strftime('%Y-%m-%d') #chatgpt
        print(forum_page, end=" | ")     
        print(topic, end=" | ")
        print(author, end=" | ")
        print(last_updated, end=" | ")
        print(f"{replies_str} replies | {num_views} views")
        
        topic_list.append(topic)
        author_list.append(author)
        date_list.append(last_updated)
        forum_page_list.append(forum_page)
        replies_list.append(replies)
        stats_list.append(num_views)
        topic_url_list.append(topic_url)
        
        firstposts_list.append(initial_post)
        initial_post_username_list.append(initial_post_username)
        initial_post_date_list.append(initial_post_date)
    
        page_topic_count=page_topic_count+1
        
    topic_by_author = pd.DataFrame(list(zip(forum_page_list, topic_list, initial_post_username_list, author_list, replies_list, stats_list, initial_post_date_list, date_list, topic_url_list, firstposts_list)), columns =['Forum Page','Topic', 'Initial Author','Author', 'Replies', 'Stats List', 'Started', 'Last Updated', 'URL', 'First Post'])
    page_topic_count_list.append(page_topic_count)
    total_pages_evaluated=total_pages_evaluated+1

In [ ]:
topic_by_author_3_to_103 = topic_by_author

# Exporting DataFrame to CSV
topic_by_author_3_to_103.to_csv('celiac_com_forum_13_p3to103.csv', index=False)  # Exporting to CSV without row indices

# Exporting DataFrame to Excel (XLSX)
topic_by_author_3_to_103.to_excel('celiac_com_forum_13_p3to103.xlsx', index=False)

In [ ]:

# Get the value of 'href' attribute
uClient2 = uReq('https://www.celiac.com/forums/topic/158655-25-years-old-15-yearof-testing-accurate-blood-tests-and-symptoms/')
page_html2 = uClient2.read()
uClient2.close()
page_soup2 = soup(page_html2, "html.parser")
posts = page_soup2.findAll("div",{"data-role":"commentContent"})
initial_post = posts[0].find_all('p')
extracted_text = ' '.join([text.get_text(strip=True) for text in initial_post])
initial_post_username = page_soup2.find("p",{"class":"ipsType_reset ipsType_blendLinks"}).a.text
initial_post_dt = page_soup2.find("p",{"class":"ipsType_reset ipsType_blendLinks"}).time.get("datetime")[0:10]
initial_post_dt_format = datetime.strptime(initial_post_dt, '%Y-%m-%d')
initial_post_date = initial_post_dt_format.date()
#print(posts)
print(extracted_text)
#firstposts_list.append(initial_post)
#initial_post_username_list.append(initial_post_username)
#initial_post_date_list.append(initial_post_date)

In [ ]:
topic_by_author_3_to_103.iloc[:,9]

In [ ]:
topic_url_list

In [ ]:

print(initial_post_username)
print(initial_post_date_dt.date())